In [2]:
import sys
sys.path.append("../../../") # Where to find utilssphering
import utilssphering
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob

In [3]:
# Read CSV
match_rep_df = pd.read_csv("../../../checkpoints/match_rep_df.csv")
match_rep_df

,Vendor,Batch,Plate_Map_Name,Assay_Plate_Barcode,Modality,Images_per_well,Sites-SubSampled,Binning,Magnification,Number_of_channels,...,Size_MB_std,channel_names,brightfield_z_plane_used,feature_channels_found,sphering,value_95_replicating,Percent_Replicating,Percent_Matching,value_95_matching,cell_count
0,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_4site_10XPA,Confocal,4,NaN,1,10,6,...,0.000144,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA",True,0.200046,57.777778,20.930233,0.303191,2014937
1,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_4site_10XPA,Confocal,4,NaN,1,10,6,...,0.000144,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA",False,0.420650,35.555556,16.279070,0.307224,2014937
2,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crest,Confocal,4,NaN,1,10,6,...,0.000183,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA",True,0.268013,62.222222,20.930233,0.401802,2413350
3,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crest,Confocal,4,NaN,1,10,6,...,0.000183,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA",False,0.338900,47.777778,16.279070,0.411854,2413350
4,MolDev,Scope1_MolDev_10X_4siteZ,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crestz,Confocal,4,NaN,1,10,6,...,0.000142,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA",True,0.205121,66.666667,23.255814,0.363114,2381443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,JUMP-MOA_compound_platemap,BRO0117059_20X,Confocal,9,4.0,1,20,6,...,0.000583,"AGP, BrightField, DNA, ER, Mito, RNA",Z08,"AGP, BrightField, DNA, ER, Mito, RNA",False,0.628379,20.000000,6.976744,0.772160,489099
358,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,JUMP-MOA_compound_platemap,BRO01177034_20x,Confocal,9,4.0,1,20,6,...,0.000014,"AGP, BrightField, DNA, ER, Mito, RNA",Z17,"AGP, BrightField, DNA, ER, Mito, RNA",True,0.139090,56.666667,18.604651,0.193171,452567
359,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,JUMP-MOA_compound_platemap,BRO01177034_20x,Confocal,9,4.0,1,20,6,...,0.000014,"AGP, BrightField, DNA, ER, Mito, RNA",Z17,"AGP, BrightField, DNA, ER, Mito, RNA",False,0.666818,21.111111,9.302326,0.790813,452567
360,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_40X_BRO0117059,JUMP-MOA_compound_platemap,BRO0117059_40x,Confocal,9,4.0,1,40,6,...,0.080201,"AGP, BrightField, DNA, ER, Mito, RNA",Z08,"AGP, BrightField, DNA, ER, Mito, RNA",True,0.193787,53.333333,20.930233,0.234702,116595


# Microscopes used by each vendor

- Yokogawa US
  - Yokogawa CV8000
- Yokogawa Japan
  - CV7000 (unsure that this is right. Need confirmation)
- Perkin Elmer
  - Opera Phenix Plus
- Molecular Devices
  - ImageXpress Micro Confocal (IXM-Confocal)
- Nikon
  - Eclipse Ti2 inverted microscope

In [20]:
moldev_cols = [
    'Percent_Score',
    'Vendor',
    'Modality',
    'Binning',
    'Magnification',
    'aperture',
    'Number_of_channels',
    'z_plane',
    'spinning-disc',
    # 'Percent_Replicating',
    # 'Percent_Matching',
    'channel_names',
    'sites'
]

df = match_rep_df[
    (match_rep_df["Vendor"].str.contains("MolDev")) &
    (~match_rep_df["Batch"].str.contains("siteSub"))
].copy()

# Create an aggregation score for replicating/matching and normalize to the max value
df["Percent_Score"] = df[["Percent_Replicating", "Percent_Matching"]].mean(axis=1)
df["Percent_Score"] = (df["Percent_Score"] / df["Percent_Score"].max()) * 100
df = df.round({"Percent_Score": 1})

df = df.sort_values("Percent_Score", ascending=False)[moldev_cols]
df["Place"] = df.reset_index(drop=True).index+1
# Move place column to first position
df.insert(0, "Place", df.pop("Place"))
df
# df.to_csv("(A)-moldev_leaderboard.csv", index=False)

,Place,Percent_Score,Vendor,Modality,Binning,Magnification,aperture,Number_of_channels,z_plane,spinning-disc,channel_names,sites
8,1,100.0,MolDev,Confocal,1,20,0.75,6,1,no,"Actin, DNA, ER, Golgi, Mito, RNA",9
4,2,98.7,MolDev,Confocal,1,10,0.45,6,3,yes,"Actin, DNA, ER, Golgi, Mito, RNA",4
2,3,91.2,MolDev,Confocal,1,10,0.45,6,1,yes,"Actin, DNA, ER, Golgi, Mito, RNA",4
10,4,90.0,MolDev,Confocal,1,20,0.75,6,1,yes,"Actin, DNA, ER, Golgi, Mito, RNA",9
0,5,86.4,MolDev,Confocal,1,10,0.45,6,1,no,"Actin, DNA, ER, Golgi, Mito, RNA",4
6,6,83.8,MolDev,Confocal,1,20,0.75,6,3,yes,"Actin, DNA, ER, Golgi, Mito, RNA",4
9,7,83.8,MolDev,Confocal,1,20,0.75,6,1,no,"Actin, DNA, ER, Golgi, Mito, RNA",9
5,8,80.3,MolDev,Confocal,1,10,0.45,6,3,yes,"Actin, DNA, ER, Golgi, Mito, RNA",4
3,9,70.3,MolDev,Confocal,1,10,0.45,6,1,yes,"Actin, DNA, ER, Golgi, Mito, RNA",4
11,10,70.2,MolDev,Confocal,1,20,0.75,6,1,yes,"Actin, DNA, ER, Golgi, Mito, RNA",9


In [64]:
yokogawa_cols = [
    'Percent_Score',
    'Vendor',
    'Modality',
    'Binning',
    'Magnification',
    'aperture',
    'Number_of_channels',
    "simultaneous-excitation",
    'z_plane',
    "dry-immersion",
    'Percent_Score',
    # 'Percent_Replicating',
    # 'Percent_Matching',
    'channel_names',
    'sites'
]

df = match_rep_df[
    (match_rep_df["Vendor"].str.contains("Yokogawa")) &
    (~match_rep_df["Batch"].str.contains("siteSub"))
].copy()

# Create an aggregation score for replicating/matching and normalize to the max value
df["Percent_Score"] = df[["Percent_Replicating", "Percent_Matching"]].mean(axis=1)
df["Percent_Score"] = (df["Percent_Score"] / df["Percent_Score"].max()) * 100
df = df.round({"Percent_Score": 1})

df = df.sort_values("Percent_Score", ascending=False)[yokogawa_cols]
df

# df.to_csv("(B)-yokogawa_leaderboard.csv", index=False)

,Percent_Score,Vendor,Modality,Binning,Magnification,aperture,Number_of_channels,simultaneous-excitation,z_plane,dry-immersion,Percent_Score,channel_names,sites
100,100.0,Yokogawa_US,Confocal,1,20,1.00,5,2.0,12,water,100.0,"AGP, DNA, ER, Mito, RNA",9
90,97.5,Yokogawa_Japan,Confocal,1,20,0.75,5,NaN,10,dry,97.5,"AGP, DNA, ER, Mito, RNA",9
98,96.4,Yokogawa_US,Confocal,1,20,1.00,5,2.0,1,water,96.4,"AGP, DNA, ER, Mito, RNA",9
102,93.8,Yokogawa_US,Confocal,1,20,1.00,6,4.0,12,water,93.8,"AGP, BrightField, DNA, ER, Mito, RNA",9
96,93.6,Yokogawa_US,Confocal,1,20,1.00,5,2.0,1,water,93.6,"AGP, DNA, ER, Mito, RNA",9
94,88.9,Yokogawa_US,Confocal,1,10,0.40,6,NaN,12,dry,88.9,"AGP, BrightField, DNA, ER, Mito, RNA",4
101,87.6,Yokogawa_US,Confocal,1,20,1.00,5,2.0,12,water,87.6,"AGP, DNA, ER, Mito, RNA",9
103,86.3,Yokogawa_US,Confocal,1,20,1.00,6,4.0,12,water,86.3,"AGP, BrightField, DNA, ER, Mito, RNA",9
104,86.2,Yokogawa_US,Confocal,1,20,1.00,6,1.0,12,water,86.2,"AGP, BrightField, DNA, ER, Mito, RNA",9
105,83.8,Yokogawa_US,Confocal,1,20,1.00,6,1.0,12,water,83.8,"AGP, BrightField, DNA, ER, Mito, RNA",9


In [61]:
nikon_cols = [
    'Percent_Score',
    'Vendor',
    'Modality',
    'Binning',
    'Magnification',
    'aperture',
    'Number_of_channels',
    'z_plane',
    "dry-immersion",
    'Percent_Score',
    # 'Percent_Replicating',
    # 'Percent_Matching',
    'channel_names',
    'sites'
]

df = match_rep_df[
    (match_rep_df["Vendor"].str.contains("Nikon")) &
    (~match_rep_df["Batch"].str.contains("siteSub"))
].copy()

# Create an aggregation score for replicating/matching and normalize to the max value
df["Percent_Score"] = df[["Percent_Replicating", "Percent_Matching"]].mean(axis=1)
df["Percent_Score"] = (df["Percent_Score"] / df["Percent_Score"].max()) * 100
df = df.round({"Percent_Score": 1})

df = df.sort_values("Percent_Score", ascending=False)[nikon_cols]

df.to_csv("(C)-nikon_leaderboard.csv", index=False)

In [62]:
pe_cols = [
    'Percent_Score',
    'Vendor',
    'Modality',
    'Binning',
    'Magnification',
    'aperture',
    'Number_of_channels',
    'z_plane',
    "dry-immersion",
    'Percent_Score',
    # 'Percent_Replicating',
    # 'Percent_Matching',
    'channel_names',
    'sites'
]

df = match_rep_df[
    (match_rep_df["Vendor"].str.contains("PE")) &
    (~match_rep_df["Batch"].str.contains("siteSub"))
].copy()

# Create an aggregation score for replicating/matching and normalize to the max value
df["Percent_Score"] = df[["Percent_Replicating", "Percent_Matching"]].mean(axis=1)
df["Percent_Score"] = (df["Percent_Score"] / df["Percent_Score"].max()) * 100
df = df.round({"Percent_Score": 1})

df = df.sort_values("Percent_Score", ascending=False)[pe_cols]

df.to_csv("(D)-pe_leaderboard.csv", index=False)